# Predicting the Super Bowl 2022 Winner

## 1. Data Wrangling

In [1]:
# Import relevat libraries

import numpy as np
import pandas as pd
from datetime import date, datetime

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# Import a sheet from source Excel file and create Pandas data frame

BUF = pd.read_excel('NFL_2021_season.xlsx', sheet_name='BUF')

In [3]:
# Check the data from imported Excel sheet

BUF.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Defense,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Expected Points,Unnamed: 23,Unnamed: 24
0,Week,Day,Date,NaN,NaN,NaN,OT,Rec,NaN,Opp,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms
1,1,Sun,2021-09-12 00:00:00,1:00PM ET,boxscore,L,NaN,0-1,NaN,Pittsburgh Steelers,...,117,1,16,252,177,75,NaN,-3.55,0.54,-5.01
2,2,Sun,2021-09-19 00:00:00,1:00PM ET,boxscore,W,NaN,2022-01-01 00:00:00,@,Miami Dolphins,...,143,2,13,223,152,71,3,8.16,24.67,-0.37


Looks like majority of the column titles are in the first row of the data frame. Let's bring them in the titles row.

In [4]:
BUF.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Score', 'Unnamed: 11', 'Offense', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Defense', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Expected Points', 'Unnamed: 23',
       'Unnamed: 24'],
      dtype='object')

Headers such as 'Offense', 'Defense', and 'Expected Points' have sub-columns that are listed in the first row. We need to be mindful of such headers while renaming the columns

In [5]:
# Change column names, remove the first row, and reset the index

BUF.columns = BUF.iloc[0].values
BUF = BUF[1:]

In [6]:
BUF.reset_index(drop=True, inplace=True)
BUF.head(2)

,Week,Day,Date,NaN,NaN,NaN,OT,Rec,NaN,Opp,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms
0,1,Sun,2021-09-12 00:00:00,1:00PM ET,boxscore,L,NaN,0-1,NaN,Pittsburgh Steelers,...,117,1,16,252,177,75,NaN,-3.55,0.54,-5.01
1,2,Sun,2021-09-19 00:00:00,1:00PM ET,boxscore,W,NaN,2022-01-01 00:00:00,@,Miami Dolphins,...,143,2,13,223,152,71,3,8.16,24.67,-0.37


Let's rename the columns so that we can understand their significance.

In [7]:
columns_list = ['Week','Day','Date','Start_Time','Boxscore','Result','Overtime','Record','Home_game','Opponent','Score','Opp_score',
                '1st_downs','Total_yards','Passing_yards','Rushing_yards','Turnovers','Opp_1st_downs','Opp_Total_yards',
                'Opp_Passing_yards','Opp_Rushing_yards','Opp_Turnovers','Offense_pts','Defense_pts','Sp_teams_pts']

In [8]:
BUF.columns = columns_list

In [9]:
BUF.head().T

,0,1,2,3,4
Week,1,2,3,4,5
Day,Sun,Sun,Sun,Sun,Sun
Date,2021-09-12 00:00:00,2021-09-19 00:00:00,2021-09-26 00:00:00,2021-10-03 00:00:00,2021-10-10 00:00:00
Start_Time,1:00PM ET,1:00PM ET,1:00PM ET,1:00PM ET,8:20PM ET
Boxscore,boxscore,boxscore,boxscore,boxscore,boxscore
Result,L,W,W,W,W
Overtime,NaN,NaN,NaN,NaN,NaN
Record,0-1,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00
Home_game,NaN,@,NaN,NaN,@
Opponent,Pittsburgh Steelers,Miami Dolphins,Washington Football Team,Houston Texans,Kansas City Chiefs


For this project, the target feature is **'Score'**, which is the points scored by the team of interest. There are several features in the data frame that do not provide any valuable information. These columns are: *'Boxscore'*, *'Record'*, and subsets of *'Expected Points'* i.e., *'Offense_pts'*, *'Defense_pts'*, and *'Sp_teams_pts'*. The last three columns are predicted values, thus can be removed.

There are few more columns that can be removed from the data frame:

*'Passing_yards'* and *'Opp_passing_yards'* - these can be calculated by subtracting total yards and rushing yards. Thus, this column does not provide additional information. On the other hand, a significant correlation between total and passing yards can affect the model. Theoretically, either passing or yards columns can be removed. However, I chose rushing yards to keep since a rushing play takes more time off the clock and has more impact on time management.

One might argue that *'Opp_score'*, i.e. points scored by the opponent can also be removed. However, I think that *Opp_score* would also dictate the play selection of both teams. Thus, it will have a significant impact on the game.

We can also drop the 'day' column as a detailed timestamp information is provided in the 'Date' column.

Since 'Result' depends on the difference between points scored by both teams, we can drop it as well.

In [10]:
# Remove the unnecessary columns from the data frame

BUF = BUF.drop(columns=['Day','Boxscore','Result','Record','Offense_pts','Defense_pts','Sp_teams_pts','Passing_yards',
                        'Opp_Passing_yards'])

In [11]:
BUF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Week               20 non-null     object
 1   Date               20 non-null     object
 2   Start_Time         19 non-null     object
 3   Overtime           2 non-null      object
 4   Home_game          9 non-null      object
 5   Opponent           20 non-null     object
 6   Score              19 non-null     object
 7   Opp_score          19 non-null     object
 8   1st_downs          19 non-null     object
 9   Total_yards        19 non-null     object
 10  Rushing_yards      19 non-null     object
 11  Turnovers          12 non-null     object
 12  Opp_1st_downs      19 non-null     object
 13  Opp_Total_yards    19 non-null     object
 14  Opp_Rushing_yards  19 non-null     object
 15  Opp_Turnovers      14 non-null     object
dtypes: object(16)
memory usage: 2.8+ KB


There are 18 columns and 21 rows in the data frame, and each column has at least one missing value in them. We need to take a look at each column and format it for further analysis.

In [12]:
# Week

BUF.Week.values

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, nan,
       'Wild Card', 'Division'], dtype=object)

The Buffalo Bills played 17 games during 2021 regular season and 2 playoff games. Thus, we can remove the rows for weeks they didn't played a game. Let's check how the data frame looks before removing certain rows.

In [13]:
BUF.iloc[:,:9]

,Week,Date,Start_Time,Overtime,Home_game,Opponent,Score,Opp_score,1st_downs
0,1,2021-09-12 00:00:00,1:00PM ET,NaN,NaN,Pittsburgh Steelers,16,23,22
1,2,2021-09-19 00:00:00,1:00PM ET,NaN,@,Miami Dolphins,35,0,21
2,3,2021-09-26 00:00:00,1:00PM ET,NaN,NaN,Washington Football Team,43,21,29
3,4,2021-10-03 00:00:00,1:00PM ET,NaN,NaN,Houston Texans,40,0,26
4,5,2021-10-10 00:00:00,8:20PM ET,NaN,@,Kansas City Chiefs,38,20,20
5,6,2021-10-18 00:00:00,8:15PM ET,NaN,@,Tennessee Titans,31,34,28
6,7,NaN,NaN,NaN,NaN,Bye Week,NaN,NaN,NaN
7,8,2021-10-31 00:00:00,1:00PM ET,NaN,NaN,Miami Dolphins,26,11,24
8,9,2021-11-07 00:00:00,1:00PM ET,NaN,@,Jacksonville Jaguars,6,9,17
9,10,2021-11-14 00:00:00,1:00PM ET,NaN,@,New York Jets,45,17,26


Week 7 was the bye week for the Bills and the row after 'week 18' contains no usable information. So, we can delete these rows. We can use the 'Start_Time' values for row deletion.

In [14]:
BUF.dropna(subset=['Start_Time'], axis=0, inplace=True)

NFL games are played under various weather and other conditions depending on the location and month. E.g. weather in southern states is warmer. Thus, games during initial week at warmer locations can have higher than desirable temperature range. On the other hand, places like Denver are at high altitudes where players can have difficulty in breathing. This may limit offensive play options and potentially scoring opportunities. Another prime example can be playoff games at Green Bay in January that are often played under extremely cold weather and thus can be low scoring.

Thus, 'Date' and game venue (depends on the oppoenent and home/away game) are important features for this analysis.

In [15]:
# Convert 'Date' column to 'Datetime' object

BUF['Date'] = pd.to_datetime(BUF['Date'], errors='coerce')
BUF['Date'].iloc[:3]

0   2021-09-12
1   2021-09-19
2   2021-09-26
Name: Date, dtype: datetime64[ns]

In [16]:
# Convert 'Start_Time' column to 'Datetime' object

BUF['Start_Time'] = pd.to_datetime(BUF['Start_Time'], errors='coerce')
BUF['Start_Time'] = BUF['Start_Time'].apply(lambda x: x.time())
BUF['Start_Time'].iloc[:3]

0    13:00:00
1    13:00:00
2    13:00:00
Name: Start_Time, dtype: object

In [17]:
BUF.iloc[:2,:]

,Week,Date,Start_Time,Overtime,Home_game,Opponent,Score,Opp_score,1st_downs,Total_yards,Rushing_yards,Turnovers,Opp_1st_downs,Opp_Total_yards,Opp_Rushing_yards,Opp_Turnovers
0,1,2021-09-12,13:00:00,NaN,NaN,Pittsburgh Steelers,16,23,22,371,117,1,16,252,75,NaN
1,2,2021-09-19,13:00:00,NaN,@,Miami Dolphins,35,0,21,314,143,2,13,223,71,3


In [18]:
# Overtime

BUF.Overtime.value_counts()

OT    2
Name: Overtime, dtype: int64

In [19]:
# Map '0' on 'NaN' and '1' on 'OT'

BUF.Overtime = BUF.Overtime.map({np.nan:0,'OT':1})
BUF.Overtime.value_counts()

0    17
1     2
Name: Overtime, dtype: int64

In [20]:
# Home_game

BUF.Home_game.value_counts()

@    9
Name: Home_game, dtype: int64

In [21]:
# Map '1' on 'NaN' and '0' on '@' since '@' represents an away game

BUF.Home_game = BUF.Home_game.map({np.nan:1, '@':0})
BUF.Home_game.value_counts()

1    10
0     9
Name: Home_game, dtype: int64

In [22]:
# Opponent

BUF.Opponent.value_counts()

New England Patriots        3
Kansas City Chiefs          2
Miami Dolphins              2
New York Jets               2
New Orleans Saints          1
Indianapolis Colts          1
Jacksonville Jaguars        1
Tampa Bay Buccaneers        1
Washington Football Team    1
Tennessee Titans            1
Carolina Panthers           1
Pittsburgh Steelers         1
Houston Texans              1
Atlanta Falcons             1
Name: Opponent, dtype: int64

All remaining columns are numerical features. Let's check if the data frame contains any missing values.

In [23]:
BUF.isnull().sum()

Week                 0
Date                 0
Start_Time           0
Overtime             0
Home_game            0
Opponent             0
Score                0
Opp_score            0
1st_downs            0
Total_yards          0
Rushing_yards        0
Turnovers            7
Opp_1st_downs        0
Opp_Total_yards      0
Opp_Rushing_yards    0
Opp_Turnovers        5
dtype: int64

Many values are missing for teams' and opponents' turnover columns. Let's look at them in detail.

In [24]:
BUF.Turnovers.values

array([1, 2, nan, 1, nan, 1, nan, 3, 2, 4, 2, 1, 1, 1, nan, 3, nan, nan,
       nan], dtype=object)

In [25]:
BUF.Opp_Turnovers.values

array([nan, 3, 3, 5, 4, 1, 2, 1, 5, nan, 1, 1, nan, 1, 2, 1, nan, 2, nan],
      dtype=object)

In both columns a value is missing when the team did not commit a turnover. Let's replace the missing values with '0'.

In [26]:
BUF.fillna(value=0, inplace=True)

Let's look at the data frame again

In [27]:
BUF

,Week,Date,Start_Time,Overtime,Home_game,Opponent,Score,Opp_score,1st_downs,Total_yards,Rushing_yards,Turnovers,Opp_1st_downs,Opp_Total_yards,Opp_Rushing_yards,Opp_Turnovers
0,1,2021-09-12,13:00:00,0,1,Pittsburgh Steelers,16,23,22,371,117,1,16,252,75,0
1,2,2021-09-19,13:00:00,0,0,Miami Dolphins,35,0,21,314,143,2,13,223,71,3
2,3,2021-09-26,13:00:00,0,1,Washington Football Team,43,21,29,481,122,0,13,290,78,3
3,4,2021-10-03,13:00:00,0,1,Houston Texans,40,0,26,450,199,1,6,109,48,5
4,5,2021-10-10,20:20:00,0,0,Kansas City Chiefs,38,20,20,436,121,0,29,392,120,4
5,6,2021-10-18,20:15:00,0,0,Tennessee Titans,31,34,28,417,82,1,16,362,146,1
7,8,2021-10-31,13:00:00,0,1,Miami Dolphins,26,11,24,351,102,0,16,262,68,2
8,9,2021-11-07,13:00:00,0,0,Jacksonville Jaguars,6,9,17,301,72,3,16,218,79,1
9,10,2021-11-14,13:00:00,0,0,New York Jets,45,17,26,489,139,2,22,366,70,5
10,11,2021-11-21,13:00:00,0,1,Indianapolis Colts,15,41,19,311,91,4,28,370,264,0


We can repeat these steps for the records of other teams and then combine all the data to create a single team statistics data frame

In [28]:
# Create a list of team names

sheet_names = ['BUF','NE','MIA','NYJ','TEN','IND','HOU','JAC','CIN','PIT','CLE','BAL','KC','LV','LAC','DEN',
             'DAL','PHI','WFT','NYG','TB','NO','ATL','CAR','GB','MIN','CHI','DET','LAR','ARI','SF','SEA']

team_names = ['Buffalo Bills','New England Patrios','Miami Dolphins','New York Jets','Tennessee Titans','Indianapolis Colts',
              'Houston Texans','Jacksonville Jaguars','Cincinnati Bengals','Pittsburgh Steelers','Cleveland Browns',
              'Baltimore Ravens','Kansas City Chiefs','Las Vegas Raiders','Los Angeles Chargers','Denver Broncos',
              'Dallas Cowboys','Philadelphia Eagles','Washington Football Team','New York Giants','Tampa Bay Buccaneers',
              'New Orleans Saints','Atlanta Falcons','Carolina Panthers','Green Bay Packers','Minnesota Vikings',
              'Chicago Bears','Detroit Lions','Los Angeles Rams','Arizona Cardinals','San Francisco 49ers','Seattle Seahawks']

In [29]:
# Import statistics of all teams from the source

team_stats = pd.DataFrame()

for team in sheet_names:
    data = pd.read_excel('NFL_2021_season.xlsx', sheet_name=team)
    i = sheet_names.index(team)
    data['Team'] = team_names[i]
    team_stats = team_stats.append(data, ignore_index=True)

In [30]:
# Explore the data frame

team_stats.head().T

,0,1,2,3,4
Unnamed: 0,Week,1,2,3,4
Unnamed: 1,Day,Sun,Sun,Sun,Sun
Unnamed: 2,Date,2021-09-12 00:00:00,2021-09-19 00:00:00,2021-09-26 00:00:00,2021-10-03 00:00:00
Unnamed: 3,NaN,1:00PM ET,1:00PM ET,1:00PM ET,1:00PM ET
Unnamed: 4,NaN,boxscore,boxscore,boxscore,boxscore
Unnamed: 5,NaN,L,W,W,W
Unnamed: 6,OT,NaN,NaN,NaN,NaN
Unnamed: 7,Rec,0-1,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00
Unnamed: 8,NaN,NaN,@,NaN,NaN
Unnamed: 9,Opp,Pittsburgh Steelers,Miami Dolphins,Washington Football Team,Houston Texans


#### Perform the steps previously performed on 'BUF' data frame

In [31]:
# Change column names, remove the first row, and reset the index

team_stats.columns = team_stats.iloc[0].values
team_stats = team_stats[1:]
team_stats.reset_index(drop=True, inplace=True)

In [32]:
# Rename the columns

columns_list = ['Week','Day','Date','Start_Time','Boxscore','Result','Overtime','Record','Home_game','Opponent','Score','Opp_score',
                '1st_downs','Total_yards','Passing_yards','Rushing_yards','Turnovers','Opp_1st_downs','Opp_Total_yards',
                'Opp_Passing_yards','Opp_Rushing_yards','Opp_Turnovers','Offense_pts','Defense_pts','Sp_teams_pts','Team']

team_stats.columns=columns_list

In [33]:
# Remove the unnecessary columns from the data frame

team_stats = team_stats.drop(columns=['Day','Boxscore','Result','Record','Offense_pts','Defense_pts','Sp_teams_pts',
                                      'Passing_yards','Opp_Passing_yards'])

In [34]:
# Remove the weeks in which each team did not play a game

team_stats.dropna(subset=['Start_Time'], axis=0, inplace=True)

In [35]:
# Explore 'Week' column

team_stats.Week.value_counts()

1               32
3               32
4               32
5               32
18              32
17              32
16              32
15              32
2               32
11              30
12              30
8               30
10              28
13              28
14              28
9               28
6               28
7               26
Wild Card       12
Division         8
Conf. Champ.     4
SuperBowl        2
Name: Week, dtype: int64

There are several weeks when all 32 teams played a game. On the other hand, 30 teams played a game during 3 weeks and 28 teams played a game during 5 weeks. Only 26 teams played a game during week 7. We don't have the result of Super Bowl yet, so let's remove the corresponding 2 rows. Also,it's convenient to rename playoff weeks in numerical order for further analysis.

In [36]:
# Rename playoff weeks and remove Super Bowl week

team_stats = team_stats[team_stats.Week != 'SuperBowl']

In [37]:
team_stats.Week = team_stats.Week.replace({'Wild Card':19,'Division':20,'Conf. Champ.':21})
team_stats.Week.unique()

array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20,  7, 21], dtype=int64)

In [38]:
# Convert 'Date' and 'Start_Time' columns to 'Datetime' objects

team_stats['Date'] = pd.to_datetime(team_stats['Date'], errors='coerce')
team_stats['Date'] = team_stats['Date'].apply(lambda x: x.date())

team_stats['Start_Time'] = pd.to_datetime(team_stats['Start_Time'], errors='coerce')
team_stats['Start_Time'] = team_stats['Start_Time'].apply(lambda x: x.time())

In [39]:
# In 'Overtime' column map '0' on 'NaN' and '1' on 'OT'

team_stats.Overtime = team_stats.Overtime.map({np.nan:0,'OT':1})
team_stats.Overtime.value_counts()

0    522
1     46
Name: Overtime, dtype: int64

In [40]:
# In 'Home_game' column map '1' on 'NaN' and '0' on '@' since '@' represents an away game

team_stats.Home_game = team_stats.Home_game.map({np.nan:1, '@':0})
team_stats.Home_game.value_counts()

0    284
1    284
Name: Home_game, dtype: int64

In [41]:
# Check data frame for missing values

team_stats.isnull().sum()

Week                   0
Date                   0
Start_Time             0
Overtime               0
Home_game              0
Opponent               0
Score                  0
Opp_score              0
1st_downs              0
Total_yards            0
Rushing_yards          0
Turnovers            171
Opp_1st_downs          0
Opp_Total_yards        0
Opp_Rushing_yards      0
Opp_Turnovers        171
Team                   0
dtype: int64

Since only turnover columns have missing values, we can fill them with '0'.

In [42]:
# Replace missing values with '0'.

team_stats.fillna(value=0, inplace=True)

In [43]:
team_stats.head(2).T

,0,1
Week,1,2
Date,2021-09-12,2021-09-19
Start_Time,13:00:00,13:00:00
Overtime,0,0
Home_game,1,0
Opponent,Pittsburgh Steelers,Miami Dolphins
Score,16,35
Opp_score,23,0
1st_downs,22,21
Total_yards,371,314


In [44]:
# Review the data frame

team_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568 entries, 0 to 646
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Week               568 non-null    int64 
 1   Date               568 non-null    object
 2   Start_Time         568 non-null    object
 3   Overtime           568 non-null    int64 
 4   Home_game          568 non-null    int64 
 5   Opponent           568 non-null    object
 6   Score              568 non-null    int64 
 7   Opp_score          568 non-null    int64 
 8   1st_downs          568 non-null    int64 
 9   Total_yards        568 non-null    int64 
 10  Rushing_yards      568 non-null    int64 
 11  Turnovers          568 non-null    int64 
 12  Opp_1st_downs      568 non-null    int64 
 13  Opp_Total_yards    568 non-null    int64 
 14  Opp_Rushing_yards  568 non-null    int64 
 15  Opp_Turnovers      568 non-null    int64 
 16  Team               568 non-null    object
dt

In [45]:
# Review data statistics

team_stats.describe()

,Week,Overtime,Home_game,Score,Opp_score,1st_downs,Total_yards,Rushing_yards,Turnovers,Opp_1st_downs,Opp_Total_yards,Opp_Rushing_yards,Opp_Turnovers
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000
mean,9.915493,0.080986,0.500000,23.038732,23.038732,20.318662,343.941901,114.649648,1.297535,20.318662,343.941901,114.649648,1.297535
std,5.580886,0.273054,0.500441,10.340682,10.340682,5.058968,85.080884,49.166622,1.191324,5.058968,85.080884,49.166622,1.191324
min,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,47.000000,-1.000000,0.000000,4.000000,47.000000,-1.000000,0.000000
25%,5.000000,0.000000,0.000000,16.000000,16.000000,17.000000,290.000000,78.750000,0.000000,17.000000,290.000000,78.750000,0.000000
50%,10.000000,0.000000,0.500000,23.000000,23.000000,21.000000,348.500000,106.000000,1.000000,21.000000,348.500000,106.000000,1.000000
75%,15.000000,0.000000,1.000000,30.000000,30.000000,23.000000,398.000000,143.000000,2.000000,23.000000,398.000000,143.000000,2.000000
max,21.000000,1.000000,1.000000,56.000000,56.000000,36.000000,575.000000,273.000000,5.000000,36.000000,575.000000,273.000000,5.000000


In [46]:
# Save the data frame as '.csv' file

team_stats.to_csv('SuperBowl_wrangling.csv',index=False)